In [ ]:
from py_wake.examples.data.iea37 import IEA37_WindTurbines, IEA37Site
from py_wake.literature import Jensen_1983

In [ ]:
site = IEA37Site()
windTurbines = IEA37_WindTurbines()

wf_model = Jensen_1983(site, windTurbines)

In [ ]:
x, y = site.initial_position.T
sim_res = wf_model(x, y)

In [ ]:
print('Total power: %f GWh'%sim_res.aep().sum().values)